# Read & Clear Data

In [1]:
import numpy as np
import pandas as pd
import feather 
import matplotlib.pyplot as plt  
from time import time
from mailerWithUtf8 import mail
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split  
from sklearn.externals import joblib
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
import os

In [2]:
def save_dataframe(df, out_filename):
   # write to csv and no index
    t0 = time()
    df.to_csv(out_filename + ".csv", index=False, encoding='utf-8')
#     df.to_csv(out_filename + ".csv", encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [3]:
file_sample_count = "3"

In [25]:
start_time = time()
path = "C:/Users/VIPLAB/Desktop/preprocess_py/marketing_analyze/0731_sample/Game/"
# filename = '0713_april_marketing_anylze_full_table_group12_select_max_min.feather' 
# filename = "0725_april_marketing_anylze_select_294_table_group11_max_min_sample_adult_others_1.csv"
filename = "0731_marketing_with_picked_group11_numeric_max_min_sample_Game_others_0.csv"

# train_df = feather.read_dataframe(path + filename)
train_df = pd.read_csv(path + filename, error_bad_lines=False)

print('load feather %s seconds', format(time() - start_time))

load feather %s seconds 5.171085596084595


In [26]:
# filename = '0713_april_marketing_anylze_full_table_group12_max_min_test.feather' 
# filename = '0713_april_marketing_anylze_full_table_group12_select_max_min_test.feather' 
# filename = '0713_april_marketing_anylze_full_table_group12_max_min.feather' 

# test_df = feather.read_dataframe(path + filename)
# filename = "0725_april_marketing_anylze_select_294_table_group11_max_min_test.csv"
# test_df = pd.read_csv(path + filename, error_bad_lines=False)

# print('load feather %s seconds', format(time() - start_time))

In [27]:
# train_df = train_df[train_df['Groups'] != 'Infrequent_User']
# test_df = test_df[test_df['Groups'] != 'Infrequent_User']


In [28]:
# train_df, test_df = train_test_split(df, test_size = 0.4, random_state = 200)


In [29]:
# train_df = train_df[wants_col]
# test_df = test_df[wants_col]

In [30]:
copy_df = train_df[["Groups"]].copy()

In [31]:
copy_df

,Groups
0,Game
1,Game
2,Game
3,Game
4,Game
5,Game
6,Game
7,Game
8,Game
9,Game


In [32]:
CLASSIFIER = "RANDOM_FOREST"
CURRENT_MODE = "adult_vs_others"


In [33]:
groups = train_df["Groups"].unique()

In [34]:
out_filename = CLASSIFIER + "_" + CURRENT_MODE + '_'

In [35]:
out_path = "clf_random_forest_model_kFold/adult_test_vs_others/" + file_sample_count +"/"
if not os.path.exists(out_path):
    os.makedirs(out_path)

In [36]:
train_x, train_y = train_df.iloc[:, 0:-1].values, train_df.iloc[:, -1].values

In [37]:
# test_x, test_y = test_df.iloc[:, 0:-1].values, test_df.iloc[:, -1].values

# convert y to numeric# 

In [38]:
le = preprocessing.LabelEncoder()
le.fit(train_df["Groups"].unique())
train_numeric_y = le.transform(train_y)


In [39]:
# test_numeric_y = le.transform(test_y) 

# build Random Forest Classifier


In [40]:
start_time = time()


In [41]:
rf = RandomForestClassifier(max_features='auto',
                            oob_score=True,
                            random_state=1,
                            n_jobs=-1,
                            n_estimators = 100)



In [42]:
param_grid = {"criterion" : ["gini"], 
              "min_samples_leaf" : [10], 
              "n_estimators": [100]}
# best 
# {'criterion': 'gini',
#  'max_depth': 25,
#  'min_samples_leaf': 25,
#  'min_samples_split': 2,
#  'n_estimators': 100}

In [43]:
# param_grid = {"criterion" : ["gini"], 
#               "min_samples_leaf" : [10], 
#               "min_samples_split" : [2],
# #               "max_depth" : [10],
#               "n_estimators": [100]}

In [44]:
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=3)
grid_clf = gs.fit(train_x, train_numeric_y)

In [45]:
gs.cv_results_["mean_test_score"][0]

0.55264295399773811

In [49]:
clf = grid_clf.best_estimator_

In [75]:
clf.tree_

AttributeError: 'RandomForestClassifier' object has no attribute 'tree_'

In [93]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
#     print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if depth > 5:
            return
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)

In [105]:
target = "Game"
target_col = "P3M_AVG_DATA_USAGE_MB"

In [106]:
train_df[train_df["Groups"] == target].loc[:, target_col].describe()

count    8.444300e+04
mean     5.759230e-03
std      8.081648e-03
min      5.371974e-07
25%      1.142888e-03
50%      3.113865e-03
75%      7.353561e-03
max      3.036825e-01
Name: P3M_AVG_DATA_USAGE_MB, dtype: float64

In [107]:
train_df[train_df["Groups"] != target].loc[:, target_col].describe()

count    84440.000000
mean         0.005982
std          0.007818
min          0.000000
25%          0.001057
50%          0.003599
75%          0.008167
max          0.277089
Name: P3M_AVG_DATA_USAGE_MB, dtype: float64

In [121]:
tree = clf[14]
# tree_to_code(tree, train_df.columns)
tree.n_features_ 
tree.max_features_ 
# tree.tree_.sample_weight

13

In [110]:

# for idx, tree in enumerate(clf):
#     if(idx == 9):
#         tree_to_code(tree, train_df.columns)
#         sample_weight

  if GENDER_CODE <= 0.25:
    if P3M_AVG_DATA_USAGE_MB <= 9.172646241495386e-05:
      if MTC_FET_DUR <= 0.0028613931499421597:
        if MTC_CHM_CNT <= 0.02581413835287094:
          if PROM_CURR_PROMOTION_SUB_TYPE <= 0.10601719468832016:
          else:  # if PROM_CURR_PROMOTION_SUB_TYPE > 0.10601719468832016
        else:  # if MTC_CHM_CNT > 0.02581413835287094
          if PROM_CURR_PROM_CODE <= 0.010369774885475636:
          else:  # if PROM_CURR_PROM_CODE > 0.010369774885475636
      else:  # if MTC_FET_DUR > 0.0028613931499421597
        if MT_STM_AMT <= 0.0004376211145427078:
          return [[  2.  11.]]
        else:  # if MT_STM_AMT > 0.0004376211145427078
          if P3M_MO_ONNET_CNT <= 0.003448275849223137:
          else:  # if P3M_MO_ONNET_CNT > 0.003448275849223137
    else:  # if P3M_AVG_DATA_USAGE_MB > 9.172646241495386e-05
      if MO_TTL_COUNT <= 0.0022878535091876984:
        if SMS_INV_AMT <= 4.040218482259661e-05:
          if P3M_MO_ONNET_DUR <= 1.1701093171

In [71]:
train_df.groupby("AGE").count()

,DATA_MONTH,RENEW_IND,GA_IND,ACTV_TYPE,SRV_TYPE_DESC,GENDER_CODE,ORIG_OPR_ID,TENURE_SCV,BILL_CITY_NAME,BILL_ZIP_CODE,...,CWC_CATG5_CNT,CWC_CATG6_CNT,CWC_CATG7_CNT,CWC_CATG8_CNT,CWC_CATG9_CNT,CWC_CATG10_CNT,CWC_CATG11_CNT,CWC_CATG12_CNT,CWC_CATG13_CNT,Groups
AGE,,,,,,,,,,,,,,,,,,,,,
0.000000,67,67,67,67,67,67,67,67,67,67,...,67,67,67,67,67,67,67,67,67,67
0.011111,112,112,112,112,112,112,112,112,112,112,...,112,112,112,112,112,112,112,112,112,112
0.022222,206,206,206,206,206,206,206,206,206,206,...,206,206,206,206,206,206,206,206,206,206
0.033333,319,319,319,319,319,319,319,319,319,319,...,319,319,319,319,319,319,319,319,319,319
0.044444,382,382,382,382,382,382,382,382,382,382,...,382,382,382,382,382,382,382,382,382,382
0.055556,626,626,626,626,626,626,626,626,626,626,...,626,626,626,626,626,626,626,626,626,626
0.066667,1157,1157,1157,1157,1157,1157,1157,1157,1157,1157,...,1157,1157,1157,1157,1157,1157,1157,1157,1157,1157
0.077778,1323,1323,1323,1323,1323,1323,1323,1323,1323,1323,...,1323,1323,1323,1323,1323,1323,1323,1323,1323,1323
0.088889,1526,1526,1526,1526,1526,1526,1526,1526,1526,1526,...,1526,1526,1526,1526,1526,1526,1526,1526,1526,1526


In [95]:
from winsound import Beep
Beep(440, 500) 
Beep(440, 500) 

In [96]:
gs.cv_results_

{'mean_fit_time': array([ 106.46937056]),
 'mean_score_time': array([ 1.10299807]),
 'mean_test_score': array([ 0.67489757]),
 'mean_train_score': array([ 0.77343137]),
 'param_criterion': masked_array(data = ['gini'],
              mask = [False],
        fill_value = ?),
 'param_max_depth': masked_array(data = [25],
              mask = [False],
        fill_value = ?),
 'param_min_samples_leaf': masked_array(data = [25],
              mask = [False],
        fill_value = ?),
 'param_min_samples_split': masked_array(data = [2],
              mask = [False],
        fill_value = ?),
 'param_min_weight_fraction_leaf': masked_array(data = [0],
              mask = [False],
        fill_value = ?),
 'param_n_estimators': masked_array(data = [500],
              mask = [False],
        fill_value = ?),
 'params': ({'criterion': 'gini',
   'max_depth': 25,
   'min_samples_leaf': 25,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0,
   'n_estimators': 500},),
 'rank_test_score': 

In [98]:
gs.best_params_

{'criterion': 'gini',
 'max_depth': 25,
 'min_samples_leaf': 25,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0,
 'n_estimators': 500}

In [99]:
gs.best_score_

0.67489757399990791

In [100]:
from mailerWithUtf8 import mail
test=mail()
test.main("clf_info finished", "done")

Start Send Mail
User Email : aa26365566@gmail.com - DONE 
Finish Send Mail


# saving classifier

In [27]:
# joblib.dump(clf, out_path + out_filename + 'CLF.pkl') 
# clf = joblib.load('filename.pkl') 


In [28]:
clf_info = str(("Accuracy on training set: %f" % gs.cv_results_["mean_test_score"][0])) + '\n'
# clf_info += str(("Accuracy on test set: %f" % clf.score(test_x, test_numeric_y))) + '\n'
clf_info += str(('fit time %s seconds' % format(time() - start_time))) + '\n'
print(clf_info)

Accuracy on training set: 0.671896
fit time 770.127076625824 seconds



In [ ]:
# predict_y = clf.predict(test_x)

In [ ]:
# predict_y 

In [ ]:
# test_numeric_y

# get feature importances

In [102]:
important_dict = dict(zip(train_df.columns[:-1],clf.feature_importances_))

In [103]:
important_dict

{'ACTV_CHANNEL_NAME': 0.0057254655176351352,
 'ACTV_TYPE': 0.0016295484742762669,
 'AGE': 0.0055614526308373387,
 'AGENCY_AMT': 0.0020701926143019754,
 'AGENCY_FLAG': 0.00060214502134019134,
 'ANOTHER_ACTIVE_D': 1.8066764197558478e-06,
 'ANOTHER_ACTIVE_VD': 4.7627279217794688e-05,
 'APPLY_CURR_EBILL_SBILL_FLAG': 0.0012596318204756004,
 'AUTOPAY_IND': 0.0011768960632969796,
 'AVG_HS_USE_MONTH': 0.0,
 'AVG_HS_USE_MONTH_EX_CURR': 0.0,
 'BEST_DRP_DESC': 0.00083424683769659825,
 'BEST_VRP_DESC': 0.00027222687843312261,
 'BILL_CITY_NAME': 0.0041715371032302545,
 'BILL_DISTRICT_NAME': 0.0073437941045640274,
 'BL_CYCLE_DESC': 0.0028529104476178015,
 'BONDING_FLAG': 0.0011317042268812448,
 'BOOSTER_AMT': 0.0,
 'CCI_CHG_DATE': 0.0,
 'CHANNEL_CHURN_INDEX': 4.8543723929085405e-05,
 'CHURN_CLM_IND': 0.00033814361777349545,
 'CHURN_IND': 3.6233729729888933e-06,
 'CHURN_INDEX': 4.6607436345834054e-05,
 'CHURN_TYPE': 7.6867719600520355e-07,
 'CHURN_WELCOME_STAGE': 4.657503063992222e-05,
 'CURR_BILL_PL

In [109]:
important_list = sorted(important_dict.items(), key=lambda x: x[1])

In [110]:
important_list.reverse()

In [112]:
clf_info += '\n\nFeature Importances\n===================\n'
for row in important_list:
    clf_info += str(row) + "\n"
    print(str(row))

('GENDER_CODE', 0.095645676306105515)
('DATA_USAGE_MB', 0.037731059345250048)
('IMEI_SMART_OS_FLAG', 0.03397587207558031)
('IMEI_MFG_NAME', 0.03304383909801413)
('IMEI_MKT_NAME', 0.032640131905697534)
('L3M_AVG_VAS_MB', 0.031992215002652064)
('P3M_AVG_DATA_USAGE_MB', 0.030938355919528641)
('DATA_RATING_USAGE_MB', 0.029700188568343309)
('L6M_AVG_VAS_MB', 0.029562157942655383)
('P6M_AVG_DATA_USAGE_MB', 0.029560864697812286)
('L1M_DATA_USAGE_MB', 0.025795612548566452)
('L2M_DATA_USAGE_MB', 0.023519049608132576)
('L3M_DATA_USAGE_MB', 0.021417569292567596)
('CURR_DEVICE_MAKER', 0.0092329494071570588)
('HAPPYGO_IND', 0.0081286367768757008)
('TENURE_SCV', 0.0078660000022569513)
('CURR_DEVICE_MODEL', 0.0074163627395630845)
('BILL_DISTRICT_NAME', 0.0073437941045640274)
('MT_TTL_DURATION', 0.0071509307499426399)
('L3M_AVG_NET_INV_AMT', 0.0070234674059891037)
('MOST_MT_DUR', 0.0068726683994159684)
('MT_STM_AMT', 0.0067805784309050996)
('P3M_MO_OFFNET_CNT', 0.0067431679182600682)
('P6M_MO_OFFNET_D

In [107]:
feature_df = pd.DataFrame(important_list, columns = ["COLUMN", "IMPORTANT_VALUE"])

In [108]:
t0 = time()
feature_df.to_csv(out_path + out_filename + "feature_important_descent.csv", index=False)
print("time for output csv file: %.2f" % (time()-t0))

PermissionError: [Errno 13] Permission denied: 'clf_random_forest_model_kFold/adult_test_vs_others/3/RANDOM_FOREST_adult_vs_others_feature_important_descent.csv'

In [ ]:
# cpy_dict = dict(important_list)
# cpy_dict["Groups"] = target_groups
# feature_df = pd.DataFrame(cpy_dict, index = [0])

In [ ]:
# cpy_dict = dict(important_list)
# # cpy_dict["Groups"] = target_groups
# feature_df = pd.DataFrame(cpy_dict, index = [0])

In [ ]:
# t0 = time()
# feature_df.to_csv(out_path + out_filename + "feature_important_one_row.csv", index=False)
# print("time for output csv file: %.2f" % (time()-t0))

In [ ]:
clf.n_classes_

In [ ]:
clf.n_outputs_

In [ ]:
predict_y = clf.predict(train_x)

In [ ]:
# cnf_matrix = confusion_matrix(test_numeric_y, predict_y )
cnf_matrix = confusion_matrix(train_numeric_y, predict_y )
cnf_matrix

In [ ]:
group_encoder = []
for idx, row in enumerate(cnf_matrix):
#     success predict
#     print(row[idx])
    current_group = str(le.inverse_transform(idx))
    group_encoder.append(current_group)
#     current_group = groups[idx]

    print(current_group)

#     idx_count_in_group = len(test_df[test_df["Groups"] == current_group])
    idx_count_in_group = len(train_df[train_df["Groups"] == current_group])

    clf_info +=  "\n\n" + str("class = %s count = [%s / %s]" % (current_group, row[idx], idx_count_in_group))
    clf_info +=  "\n\n" + str("predict %s accurancy = %s" % (current_group, row[idx] / idx_count_in_group))
    print("class = %s count = [%s / %s]" % (current_group,row[idx],str(idx_count_in_group)))
    print("predict %s accurancy = %s" % (current_group, row[idx] / idx_count_in_group))
    print()

In [ ]:
cnf_df = pd.DataFrame(cnf_matrix)
cnf_df.columns = group_encoder
cnf_df.index = group_encoder

In [ ]:
cnf_df

In [ ]:
cnf_df.to_csv(out_path + out_filename + "confusion_matrix.csv", index=False)


In [ ]:
md_info = clf_info.replace("\n", "<br>")
with open(out_path + out_filename + 'readme.md', 'w+') as f:
     f.write(md_info)
f.closed

In [ ]:
plt.plot(clf.feature_importances_, 'o')  
# for i in 
plt.xticks(range(train_x.shape[1]), train_df.columns[:-1], rotation=90)  
plt.ylim(0, 1)  
plt.show()  

In [ ]:
from winsound import Beep
Beep(440, 500) 
Beep(440, 500) 

In [ ]:
from mailerWithUtf8 import mail
mail_info = clf_info.replace("\n", "<br>")
test=mail()
test.main("clf_info finished", mail_info)

# Check feature importances accurancy

In [113]:
feature_cols = []
limit_importances = 20
for idx, feature in enumerate(important_list):
    if(idx == limit_importances):
        break

    #     print(feature[0])
    feature_cols.append(feature[0])
feature_cols.append("Groups")

In [114]:
feature_cols

['GENDER_CODE',
 'DATA_USAGE_MB',
 'IMEI_SMART_OS_FLAG',
 'IMEI_MFG_NAME',
 'IMEI_MKT_NAME',
 'L3M_AVG_VAS_MB',
 'P3M_AVG_DATA_USAGE_MB',
 'DATA_RATING_USAGE_MB',
 'L6M_AVG_VAS_MB',
 'P6M_AVG_DATA_USAGE_MB',
 'L1M_DATA_USAGE_MB',
 'L2M_DATA_USAGE_MB',
 'L3M_DATA_USAGE_MB',
 'CURR_DEVICE_MAKER',
 'HAPPYGO_IND',
 'TENURE_SCV',
 'CURR_DEVICE_MODEL',
 'BILL_DISTRICT_NAME',
 'MT_TTL_DURATION',
 'L3M_AVG_NET_INV_AMT',
 'Groups']

In [115]:
feature_train_x = train_df[feature_cols].iloc[:, 0:-1].values
# feature_test_x = test_df[feature_cols].iloc[:, 0:-1].values

In [116]:
gs_feature = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_feature_clf = gs_feature.fit(feature_train_x, train_numeric_y)

In [122]:
clf_feature = grid_feature_clf.best_estimator_
clf_feature_info = str(("Accuracy on training set: %f" % clf_feature.score(feature_train_x, train_numeric_y))) + '\n'
# clf_feature_info += str(("Accuracy on test set: %f" % clf_feature.score(feature_test_x, test_numeric_y))) + '\n'
clf_feature_info += str(('fit time %s seconds' % format(time() - start_time))) + '\n'
clf_feature_info += str(('feature selection numbers = %s' % str(limit_importances) + '\n'))
print(clf_feature_info)

Accuracy on training set: 0.751673
fit time 2601.451102733612 seconds
feature selection numbers = 20



In [123]:
gs.cv_results_

{'mean_fit_time': array([ 106.46937056]),
 'mean_score_time': array([ 1.10299807]),
 'mean_test_score': array([ 0.67489757]),
 'mean_train_score': array([ 0.77343137]),
 'param_criterion': masked_array(data = ['gini'],
              mask = [False],
        fill_value = ?),
 'param_max_depth': masked_array(data = [25],
              mask = [False],
        fill_value = ?),
 'param_min_samples_leaf': masked_array(data = [25],
              mask = [False],
        fill_value = ?),
 'param_min_samples_split': masked_array(data = [2],
              mask = [False],
        fill_value = ?),
 'param_min_weight_fraction_leaf': masked_array(data = [0],
              mask = [False],
        fill_value = ?),
 'param_n_estimators': masked_array(data = [500],
              mask = [False],
        fill_value = ?),
 'params': ({'criterion': 'gini',
   'max_depth': 25,
   'min_samples_leaf': 25,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0,
   'n_estimators': 500},),
 'rank_test_score': 

In [124]:
gs.best_score_

0.67489757399990791

In [ ]:
predict_feature_y = clf_feature.predict(feature_test_x)

In [ ]:
cnf_matrix = confusion_matrix(test_numeric_y, predict_feature_y )
cnf_matrix

In [ ]:
group_encoder = []
for idx, row in enumerate(cnf_matrix):
#     success predict
#     print(row[idx])
    current_group = str(le.inverse_transform(idx))
    group_encoder.append(current_group)
#     current_group = groups[idx]

    print(current_group)

    idx_count_in_group = len(test_df[test_df["Groups"] == current_group])
    clf_feature_info +=  "\n\n" + str("class = %s count = [%s / %s]" % (current_group, row[idx], idx_count_in_group))
    clf_feature_info +=  "\n\n" + str("predict %s accurancy = %s" % (current_group, row[idx] / idx_count_in_group))
    print("class = %s count = [%s / %s]" % (current_group,row[idx],str(idx_count_in_group)))
    print("predict %s accurancy = %s" % (current_group, row[idx] / idx_count_in_group))
    print()

In [ ]:
cnf_pd = pd.DataFrame(cnf_matrix)
cnf_pd.columns = group_encoder
cnf_pd.index = group_encoder

In [ ]:
cnf_pd

In [ ]:
from winsound import Beep
Beep(440, 500) 
Beep(440, 500) 
Beep(440, 500) 
Beep(349, 350) 
Beep(523, 150) 
Beep(440, 500) 
Beep(349, 350) 
Beep(523, 150) 
Beep(440, 1000) 
Beep(659, 500) 
Beep(659, 500) 
Beep(659, 500) 
Beep(698, 350) 
Beep(523, 150) 
Beep(415, 500) 
Beep(349, 350) 
Beep(523, 150) 
Beep(440, 1000)